# PyTorch2.0 小练习

In [1]:
import torch
import numpy as np

## 实现softmax函数

In [4]:
def softmax(x):
    ##########
    '''实现softmax函数，只要求对最后一维归一化，
    不允许用tf自带的softmax函数'''
    ##########
    exp_x = torch.exp(x)
    prob_x = exp_x / torch.sum(exp_x, dim=-1, keepdim=True)
    return prob_x


test_data = np.random.normal(size=[10, 5])
(softmax(torch.tensor(test_data)).numpy() - torch.nn.Softmax(dim=-1)(torch.tensor(test_data)).numpy())**2 < 0.0001

True

## 实现sigmoid函数

In [7]:
def sigmoid(x):
    ##########
    '''实现sigmoid函数， 不允许用tf自带的sigmoid函数'''
    ##########
    prob_x = 1 / (1 + np.exp(-x))
    return prob_x

test_data = np.random.normal(size=[10, 5])
(sigmoid(torch.tensor(test_data)).numpy() - torch.nn.Sigmoid()(torch.tensor(test_data)).numpy())**2 < 0.0001

True

## 实现 softmax 交叉熵loss函数

In [67]:
def softmax_ce(x, label):
    ##########
    '''实现 softmax 交叉熵loss函数， 不允许用tf自带的softmax_cross_entropy函数'''
    ##########
    # / x.shape[0] 因为 torch.nn.CrossEntropyLoss() 默认reduction='mean'
    loss = -torch.sum(label * torch.log(x + 1e-10), dim=-1) / x.shape[0]
    loss = torch.mean(loss)
    return loss

test_data = np.random.normal(size=[10, 5])
prob = torch.nn.Softmax(dim=-1)(torch.tensor(test_data))
label = np.zeros_like(test_data)
label[np.arange(10), np.random.randint(0, 5, size=10)] = 1.

((torch.nn.CrossEntropyLoss()(torch.tensor(test_data), torch.tensor(label)) / label.shape[0] - softmax_ce(prob, torch.tensor(label)))**2 < 0.0001).numpy()

array(True)

## 实现 sigmoid 交叉熵loss函数

In [77]:
def sigmoid_ce(x, label):
    ##########
    '''实现 softmax 交叉熵loss函数， 不允许用tf自带的softmax_cross_entropy函数'''
    ##########
    loss = -torch.sum(label * torch.log(x + 1e-10) + (1 - label) * torch.log(1 - x + 1e-10), dim=-1) / x.shape[0]
    loss = torch.mean(loss)
    return loss

test_data = np.random.normal(size=[10]).astype(np.float32)
prob = torch.nn.Sigmoid()(torch.LongTensor(test_data))
label = np.random.randint(0, 2, 10).astype(test_data.dtype)

print(label)


((torch.nn.BCELoss()(prob, torch.tensor(label)) - sigmoid_ce(prob, torch.tensor(label)))**2 < 0.0001).numpy()

[1. 1. 0. 0. 1. 1. 1. 0. 0. 0.]


array(True)

torch实现sigmoid_cross_entropy_with_logits的方法
因为torch.nn.CrossEntropyLoss()相当于把torch.nn.LogSoftmax()作为torch.nn.NLLoss()输入
得到的prob = torch.nn.Sigmoid()(torch.tensor(test_data))输入torch.nn.CrossEntropyLoss()后又会进行LogSoftmax操作
因此我们需要将prob输入BCELoss从而模拟sigmoid_cross_entropy_with_logits